# Strategy Actor-Critic under partial Observability

> CRLD actor-critic agents learning under partial observability in strategy space  

In [ ]:
#| default_exp Agents/POStrategyActorCritic

In [ ]:
#| hide
# Imports for the nbdev development environment
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import jax
from jax import jit
import jax.numpy as jnp

import numpy as np
import itertools as it
from functools import partial

from fastcore.utils import *

from pyCRLD.Agents.POStrategyBase import POstrategybase
from pyCRLD.Utils.Helpers import *

In [ ]:
#| export
class POstratAC(POstrategybase):
    """
    Class for
    deterministic policy-average independent (multi-agent) partially observable
    temporal-difference actor-critic reinforcement learning in policy space.
    """
    
    @partial(jit, static_argnums=(0,2))
    def TDerror(self, X, norm=False):
        """
        TD error for partially observable policy AC dynamics,
        given joint policy X
        """
        Bios = self.fast_Bios(X)  # for speed up
        Xisa = self.Xisa(X)  # for speed up
        
        R = self.Rioa(X, Bios=Bios, Xisa=Xisa)
        Vio = self.Vio(X, Bios=Bios, Xisa=Xisa, Rioa=R)
        NextV = self.NextVioa(X, Bios=Bios, Xisa=Xisa, Vio=Vio)

        n = jnp.newaxis
        TDe = self.pre[:,n,n]*R + self.gamma[:,n,n]*NextV - Vio[:,:,n]
        TDe *= self.beta[:,n,n]

        TDe = TDe - TDe.mean(axis=2, keepdims=True) if norm else TDe
        return TDe
    
    @partial(jit, static_argnums=0)
    def NextVioa(self, X, Xisa=None, Bios=None, Vio=None, 
                 Tioo=None, Rio=None, Rioa=None):       
        """
        Policy-average next value for agent i, current obs o and act a.
        """
        Xisa = self.Xisa(X) if Xisa is None else Xisa
        Bios = self.fast_Bios(X) if Bios is None else Bios
        Vio = self.Vio(X, Rio=Rio, Tioo=Tioo, Bios=Bios, Xisa=Xisa, Rioa=Rioa)\
            if Vio is None else Vio
        
        i = 0; a = 1; s = 2; s_ = 3; o = 4; o_ = 5  # next observatio 
        j2k = list(range(6, 6+self.N-1))  # other agents
        b2d = list(range(6+self.N-1, 6+self.N-1 + self.N))  # all actions
        e2f = list(range(5+2*self.N, 5+2*self.N + self.N-1))  # all other acts

        sumsis = [[j2k[l], s, e2f[l]] for l in range(self.N-1)]  # sum inds
        otherY = list(it.chain(*zip((self.N-1)*[Xisa], sumsis)))
            
        args = [self.Omega, [i]+j2k+[a]+b2d+e2f, Bios, [i, o, s]] + otherY +\
               [self.T, [s]+b2d+[s_], self.O, [i, s_, o_], 
                Vio, [i, o_], [i, o, a]]
        return jnp.einsum(*args, optimize=self.opti)


In [ ]:
show_doc(POstratAC.TDerror)

---

[source](https://github.com/wbarfuss/pyCRLD/blob/main/pyCRLD/Agents/POStrategyActorCritic.py#L29){target="_blank" style="float:right; font-size:smaller"}

### POstratAC.TDerror

>      POstratAC.TDerror (X, norm=False)

TD error for partially observable policy AC dynamics,
given joint policy X

In [ ]:
show_doc(POstratAC.NextVioa)

---

[source](https://github.com/wbarfuss/pyCRLD/blob/main/pyCRLD/Agents/POStrategyActorCritic.py#L49){target="_blank" style="float:right; font-size:smaller"}

### POstratAC.NextVioa

>      POstratAC.NextVioa (X, Xisa=None, Bios=None, Vio=None, Tioo=None,
>                          Rio=None, Rioa=None)

Policy-average next value for agent i, current obs o and act a.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()